<a href="https://colab.research.google.com/github/sadrireza/Neural-Networks/blob/main/OCCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  BatchNormalization, LeakyReLU,Dropout,ReLU
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import accuracy_score

In [7]:
!ls /content

ACCURACY_OCCNN.py  FEATURES_COMP.npy	  FEATURE_TRAINING_COMP.npy  sample_data
ANOMALY_Z.npy	   FEATURE_TEST_COMP.npy  OCCNN_3.py		     Usermanual.docx


In [8]:
# CARICAMENTO DATI
training = np.load("FEATURE_TRAINING_COMP.npy")
test = np.load("FEATURE_TEST_COMP.npy")
X_Z = np.load("ANOMALY_Z.npy")

In [9]:
# CREAZIONE LABEL
X = np.append(training,X_Z,axis=0)
L_1 = [1,0]
L_2 = [0,1]
Label_1 = [L_1 for i in range(len(training))]
Label_2 = [L_2 for i in range(len(X_Z))]
Label_1 = np.array(Label_1)
Label_2 = np.array(Label_2)
Label = np.append(Label_1, Label_2, axis=0)
hidden_layer = 50

In [10]:
# CREAZIONE MODELLO
tf.keras.backend.clear_session()
model = Sequential(
    [
        Dense(hidden_layer,input_shape=(3,), activation="relu", name="Input_Layer"),
        Dense(hidden_layer, activation="relu", name="layer2"),
        Dense(2,activation="softmax", name="Last_layer"),
    ]
)

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(
    loss='binary_crossentropy',
    metrics=['accuracy'],
    optimizer=opt,
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Dense)         (None, 50)                200       
                                                                 
 layer2 (Dense)              (None, 50)                2550      
                                                                 
 Last_layer (Dense)          (None, 2)                 102       
                                                                 
Total params: 2852 (11.14 KB)
Trainable params: 2852 (11.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
#PRIMO ADDESTRAMENTO
kf = KFold(n_splits=5,shuffle=True)
kf.split(X)
accuracy_model = []
z = 0
for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Label[train_index],Label[test_index]
    # Train the model
    print(X_train.shape)
    train_report = model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), verbose=0)
    # Append to accuracy_model the accuracy of the model
    print("******Evaluate on test data with the model:**********",z)
    z = z+1
    results = model.evaluate(X_test, y_test)
    print("test loss, test acc:", results)
    print(train_report.history.keys())

(6720, 3)
******Evaluate on test data with the model:********** 0
53/53 [==============================] - 0s 2ms/step - loss: 0.0736 - accuracy: 0.9768
test loss, test acc: [0.07361356168985367, 0.9767857193946838]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6720, 3)
******Evaluate on test data with the model:********** 1
53/53 [==============================] - 0s 2ms/step - loss: 0.0639 - accuracy: 0.9815
test loss, test acc: [0.06387442350387573, 0.9815475940704346]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6720, 3)
******Evaluate on test data with the model:********** 2
53/53 [==============================] - 0s 3ms/step - loss: 0.0758 - accuracy: 0.9810
test loss, test acc: [0.07584400475025177, 0.9809523820877075]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6720, 3)
******Evaluate on test data with the model:********** 3
53/53 [==============================] - 0s 3ms/step - loss: 0.0783 - accuracy: 0.9786
test loss, test acc

In [12]:
# creazione label test

n_good = 4922-3400
n_bad = len(test)-n_good
Lable_1 = [1,0]
Lable_2 = [0,1]
Label_1 = [Lable_1 for i in range(n_good)]

Label_2 = [Lable_2 for i in range(n_bad)]
Label_1 = np.array(Label_1)
Label_2 = np.array(Label_2)
Label = np.append(Label_1, Label_2, axis=0)
model.evaluate(test, Label)

71/71 [==============================] - 0s 2ms/step - loss: 0.0865 - accuracy: 0.9636


[0.08650083839893341, 0.9635717272758484]

In [13]:
#ciclo 2
P_pred = model.predict(X_Z)
Adversal = X_Z[P_pred[:,0]<0.5,:]
len(Adversal)

#fig = plt.figure(figsize=(10, 10))
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(Adversal[:,0], Adversal[:,1], Adversal[:,2], alpha=0.5, color='red')
#ax.set_xlabel('f1')
#ax.set_ylabel('f2')
#ax.set_zlabel('TDT2')
#ax.set_xlim(-1, 1)
#ax.set_ylim(-1, 1)
#ax.set_zlim(-1, 1)
#ax.set_title('Feature space')
#plt.show()


157/157 [==============================] - 0s 2ms/step


4873

In [14]:
#preparazione training secondo ciclo
X = np.append(training,Adversal,axis=0)
L_1 = [1, 0]
L_2 = [0, 1]
Label_1 = [L_1 for i in range(len(training))]
Label_2 = [L_2 for i in range(len(Adversal))]
Label_1 = np.array(Label_1)
Label_2 = np.array(Label_2)
Label = np.append(Label_1, Label_2, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, Label, test_size=0.3)

kf = KFold(n_splits=5,shuffle=True)
kf.split(X)
accuracy_model = []
z = 0
for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Label[train_index],Label[test_index]
    # Train the model
    print(X_train.shape)
    train_report = model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), verbose=0)
    # Append to accuracy_model the accuracy of the model
    print("******Evaluate on test data with the model:**********",z)
    z = z+1
    results = model.evaluate(X_test, y_test)
    print("test loss, test acc:", results)
    print(train_report.history.keys())

model.save('OCC_50_50_2C')

(6618, 3)
******Evaluate on test data with the model:********** 0
52/52 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 0.9952
test loss, test acc: [0.013337988406419754, 0.9951661825180054]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6618, 3)
******Evaluate on test data with the model:********** 1
52/52 [==============================] - 0s 2ms/step - loss: 0.0097 - accuracy: 0.9970
test loss, test acc: [0.009712550789117813, 0.9969788789749146]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6618, 3)
******Evaluate on test data with the model:********** 2
52/52 [==============================] - 0s 2ms/step - loss: 0.0094 - accuracy: 0.9964
test loss, test acc: [0.009427186101675034, 0.9963746070861816]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6619, 3)
******Evaluate on test data with the model:********** 3
52/52 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 0.9982
test loss, test 